In [6]:
#3CTU
#Chandigar
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from decimal import Decimal
import pandas as pd
import time


driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20) 

bus_route_link = []
bus_route_name = []
bus_route_final = []
#bus_route = []
bus_names = [] 
bus_type = [] 
departing_time = []
duration = [] 
Reaching_time = [] 
star_rating = [] 
price = [] 
seat_availablity = []

def jump_pages():
    for page_number in range(1,4):
        find_bus_routes = driver.find_elements(By.CLASS_NAME,'route')
        if page_number < 4:
            for route in find_bus_routes:
                route_name = route.get_attribute("title")
                route_link = route.get_attribute("href")
                bus_route_name.append(route_name)
                bus_route_link.append(route_link)
                #print(route_name,route_link)
            try:
                pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "DC_117_paginationTable")]')))
                next_page_button=pagination_container.find_element(By.XPATH, f'//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
                action = ActionChains(driver)
                action.move_to_element(next_page_button).perform()
                time.sleep(2)
                next_page_button.click()
            except:
                print('failure')

def scroll(): # function to scroll page from top to bottom
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")# Scroll down to the bottom in order to load all the buses
        time.sleep(2)# Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
        if new_height == last_height:# If it is the same height then it is at the end of the page
            break
        last_height = new_height

def click_button(): # function to click button for view buses for government buses
    view_buses = driver.find_elements(By.CSS_SELECTOR, '.p-left-10.icon.icon-down')
    if view_buses:
        button_count = len(view_buses)
        for i in range(button_count-1 , -1, -1):
            view_buses[i].click()

def get_bus_details(): # function to get all bus details
    rate = driver.find_elements(By.CSS_SELECTOR,".fare.d-block")#
    seat_available = driver.find_elements(By.CSS_SELECTOR,".column-eight.w-15.fl")
    rating = driver.find_elements(By.CSS_SELECTOR,".column-six.p-right-10.w-10.fl")
    names = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
    b_type = driver.find_elements(By.CSS_SELECTOR,".bus-type.f-12.m-top-16.l-color.evBus")
    start_time = driver.find_elements(By.CSS_SELECTOR,".dp-time.f-19.d-color.f-bold")
    total_time = driver.find_elements(By.CSS_SELECTOR,".dur.l-color.lh-24")
    end_time =  driver.find_elements(By.CSS_SELECTOR,".bp-time.f-19.d-color.disp-Inline")
    
    time.sleep(5)
    print(len(names),len(b_type),len(start_time),len(total_time),len(end_time),len(rating),len(rate),len(seat_available))

    for i in range(0,len(seat_available)):#(len(rating)):
        bus_names.append(names[i].text)
        bus_type.append(b_type[i].text)
        departing_time.append(start_time[i].text)
        duration.append(total_time[i].text)
        Reaching_time.append(end_time[i].text)
        if (rating[i].text) == "New":
            star_rating.append(0.0)
        else:
            star_rating.append((rating[i].text)[0:3])
        price.append((rate[i].text))
        seat_availablity.append((seat_available[i].text)[0:2])
    
    print(len(bus_names),len(bus_type),len(departing_time),len(duration),len(Reaching_time),len(star_rating),len(price),len(seat_availablity))


driver.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")
driver.maximize_window()
time.sleep(5)

jump_pages()
    
print("Total routes",len(bus_route_name))

for i in range (0,len(bus_route_link)):# used 2 just to check if every thing is working as expected
    driver.get(bus_route_link[i])
    try:
        body = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.clearfix.bus-item')))
    except:
        time.sleep(1)
    click_button()
    scroll()
    time.sleep(5)
    get_bus_details()
    #print(i,"Route completed")
    temp_name = []
    temp_link = []
    bus_route = []
    state_service = []
    print(len(bus_names))
    for j in range(len(bus_names)):
        temp_name.append(bus_route_name[i])
        temp_link.append(bus_route_link[i])
        state_service.append("CTU")
    print(len(temp_name),len(temp_link))
    bus_route = list(zip(state_service,temp_name,temp_link,bus_names,bus_type,departing_time,duration,Reaching_time,star_rating,price,seat_availablity))
    bus_route_final.extend(bus_route)
    print(i+1,bus_route_name[i],"completed")
    temp_name.clear()
    temp_link.clear()
    state_service.clear()
    bus_route.clear()
    bus_names.clear()
    bus_type.clear()
    departing_time.clear()
    duration.clear()
    Reaching_time.clear()
    star_rating.clear()
    price.clear()
    seat_availablity.clear()
    
df_bus_details_3 = pd.DataFrame(bus_route_final, columns=["Service Name","Route_name", "Route_link", "Bus_names","Bus_type","Departing_time","Duration","Reaching_time","Star_rating","Price","Seat_availablity"])
df_bus_details_3['Seat_availablity']=df_bus_details_3['Seat_availablity'].astype(int)
df_bus_details_3['Price'] = df_bus_details_3['Price'].str.replace('INR ', '', regex=False)
df_bus_details_3['Star_rating'] = df_bus_details_3['Star_rating'].str.replace(' ', '', regex=False)
df_bus_details_3['Price'] = df_bus_details_3['Price'].astype(float)
df_bus_details_3['Star_rating'] = df_bus_details_3['Star_rating'].fillna(0.0)

failure
Total routes 29
307 307 307 307 307 307 307 307
307 307 307 307 307 307 307 307
307
307 307
1 Delhi to Chandigarh completed
134 134 134 134 134 134 134 134
134 134 134 134 134 134 134 134
134
134 134
2 Chandigarh to Delhi completed
12 12 12 12 12 12 12 12
12 12 12 12 12 12 12 12
12
12 12
3 Yamuna Nagar to Chandigarh completed
43 43 43 43 43 43 43 43
43 43 43 43 43 43 43 43
43
43 43
4 Ludhiana to Chandigarh completed
12 12 12 12 12 12 12 12
12 12 12 12 12 12 12 12
12
12 12
5 Chandigarh to Yamuna Nagar completed
12 12 12 12 12 12 12 12
12 12 12 12 12 12 12 12
12
12 12
6 Chandigarh to Hamirpur (Himachal Pradesh) completed
3 3 3 3 3 3 3 3
3 3 3 3 3 3 3 3
3
3 3
7 Chandigarh to Baijnath completed
10 10 10 10 10 10 10 10
10 10 10 10 10 10 10 10
10
10 10
8 Hamirpur (Himachal Pradesh) to Chandigarh completed
41 41 41 41 41 41 41 41
41 41 41 41 41 41 41 41
41
41 41
9 Chandigarh to Ludhiana completed
15 15 15 15 15 15 15 15
15 15 15 15 15 15 15 15
15
15 15
10 Chandigarh to Dehradun comple

In [14]:
#df_bus_details_3['Star_rating'] = df_bus_details_3['Star_rating'].astype(float)
df_bus_details_3.to_csv("Chandigar.csv",index = False)

In [12]:
pd.set_option('display.max_rows',None)
df_bus_details_3

,Service Name,Route_name,Route_link,Bus_names,Bus_type,Departing_time,Duration,Reaching_time,Star_rating,Price,Seat_availablity
0,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,RSRTC - 197401,Express Non AC Seater 2+3,12:20,05h 25m,17:45,3.6,320.00,42
1,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,RSRTC - 178237,Express Non AC Seater 2+3,13:10,05h 05m,18:15,3.6,326.00,42
2,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,RSRTC - 183801,Express Non AC Seater 2+3,15:30,05h 30m,21:00,3.6,320.00,42
3,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,NueGo,A/C Seater (2+2),06:00,05h 15m,11:15,3.9,333.00,21
4,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,NueGo,Electric A/C Seater (2+2),06:05,05h 20m,11:25,3.8,428.00,13
5,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),06:30,05h 00m,11:30,3.4,499.00,10
6,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,NueGo,A/C Seater (2+2),07:00,05h 15m,12:15,4.0,333.00,35
7,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,NueGo,Electric A/C Seater (2+2),07:20,05h 35m,12:55,3.9,285.00,28
8,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,NueGo,A/C Seater (2+2),08:00,04h 55m,12:55,4.0,333.00,32
9,CTU,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,City Land Travels,Volvo A/C Semi Sleeper (2+2),09:40,05h 20m,15:00,4.2,545.00,33
